In [2]:
%pip install wandb
%pip install matplotlib
%pip install numpy
%pip install tqdm
%matplotlib inline
%pip install gymnasium==0.29.1

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
#@title Imports
from collections import defaultdict #for accessing keys which are not present in dictionary
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import gymnasium as gym
import sys
import random
from matplotlib.patches import Patch
import seaborn as sns

In [4]:
class MC_BlackjackAgent:
    def __init__(self, epsilon):
        self.epsilon = epsilon #exploration rate
        self.possible_actions = [0,1] #0 for stick, 1 for hit

    def play(self, obs, best_action): #best_action is the action with highest Q value
        
        if obs[0] < 12: #if sum of player's cards is less than 12 then always hit 
            return 1 #hit

        elif obs[0] >= 12 and obs[0] < 21: #if sum of player's cards is between 12 and 21
            if np.random.random() > self.epsilon:
                action_to_take = best_action
            else:
                action_to_take = random.choice(self.possible_actions)
            return action_to_take
        
        else: #if sum of player's cards is greater or equal than 21 then always stick
            return 0

In [5]:
epsilon = 0.15
agent = MC_BlackjackAgent(epsilon)


In [9]:
from collections import deque
from gymnasium.wrappers import RecordEpisodeStatistics
from IPython.display import clear_output
import wandb
import pygame
from numpy import random

#load the environment
env = gym.make('Blackjack-v1',sab=False, natural=True, render_mode='rgb_array') #We are not folllowing the default sutton and barto book settings, which are sab=True, natural=False, render_mode='human'

possible_states = [(i, j, k) for i in range(12, 21) for j in range(1, 11) for k in range(2)] #possible states are (player_sum, dealer_card, usable_ace)
possible_actions = [0, 1] #possible actions are (stick, hit)

#dictionary for storing the action value functions for each state
dict_state_action_value_functions = {state: 
                                        {action: 
                                            {"return_sum": 0, "count":0, 'running_expected_return': 0}
                                        for action in possible_actions} 
                                    for state in possible_states} 

best_action = random.choice([0,1]) #best_action is the action with highest Q value, by default is random (only for the first iteration)

for _ in range(1):
    # Initialize wandb
    wandb.init(project="blackjack_MC_First_Visit", entity="ai42")
    pygame.init()
    
    n_episodes = 10000  # Define the number of episodes you want to run

    wins = 0.0
    losses = 0.0
    draws = 0.0
    naturals = 0.0

    for episode in tqdm(range(n_episodes)):
        obs, info = env.reset()
        terminated, truncated = False, False
        clear_output()
        step = 0
        episode_rewards = 0  # Initialize total rewards for the episode


        while not terminated and not truncated:
            action = agent.play(obs, best_action)  # Agent's policy
            
            obs, reward, terminated, truncated, info = env.step(action)
            if obs[0] >= 12 and obs[0] < 21: #actually policy only affects if sum of player's cards is between 12 and 21, then, restricting this condition we can reduce the number of iterations and make the training faster
                dict_state_action_value_functions[obs][action]["count"] += 1 #increment the count of the state-action pair
                dict_state_action_value_functions[obs][action]["return_sum"] += reward #increment the return sum of the state-action pair
                dict_state_action_value_functions[obs][action]["running_expected_return"] = dict_state_action_value_functions[obs][action]["return_sum"] / dict_state_action_value_functions[obs][action]["count"] #calculate the running expected return of the state-action pair
                
                best_action = np.argmax([dict_state_action_value_functions[obs][action]["running_expected_return"] for action in possible_actions]) #best_action is the action with highest Q value
            
            frame = env.render() # Render the frame
            step += 1 # Increment the step counter
            episode_rewards += reward  # Accumulate rewards

            # Plot frame
            plt.imshow(frame)
            plt.axis('off')
            plt.title(f"Episode: {episode} - Step: {step} - Action Taken: {action} - Reward: {reward} - Terminated: {terminated}")
            plt.savefig('frame.png')
            plt.close()

            
            
            # Log the frame and rewards to wandb
            wandb.log({
                "episode": episode,
                "step": step,
                "frame": wandb.Image('frame.png'),
                "reward": reward,
                "cumulative_reward": episode_rewards,
            })
        if reward == 1 or reward == 1.5:
            wins += 1
        elif reward == -1:
            losses += 1
        elif reward == 0:
            draws += 1
        if reward == 1.5:
            naturals += 1

    env.close()

    # Let´s log general statistics of the training
    wandb.log({"Win_rate": wins / n_episodes, "Loss_rate": losses / n_episodes, "Draw_rate": draws / n_episodes, "Natural_win_rate": naturals / n_episodes, "epsilon": epsilon}) # Log the episode statistics to wandb


100%|██████████| 10000/10000 [49:03<00:00,  3.40it/s]
